# MapsTo in pySBOL2
The MapsTo class in the Synthetic Biology Open Language (SBOL) is used to explicitly state that two `ComponentInstance` objects, often from different levels of design hierarchy, represent the same biological entity. It is most often used when `ModuleDefinition` and `ComponentDefinition` objects are composed using `Module` and `ComponentInstance` objects.

`MapsTo` objects define how a `ComponentInstance` in a higher-level design relates to a `ComponentInstance` in a lower-level design through  and refinement relationships.

MapsTo objects have the following required properties:

* `local`: Refers to the `ComponentInstance` in the higher-level design.
* `remote`: Refers to the `ComponentInstance` in the lower-level design. The referenced instance must have access="public".
* `refinement`: Specifies how to interpret the relationship between the local and remote instances using a URI. For * `example`: http://sbols.org/v2#useRemote.

This example demonstrates linking a FunctionalComponent in a high-level toggle switch module to one in a lower-level LacI inverter using a MapsTo object. We will:

1. Define all necessary biological parts (`ComponentDefinition`): two proteins (LacI, cI) and two promoters (pLac, pR).
2. Define two `ModuleDefinition` objects for genetic inverters: one repressed by LacI and one repressed by cI.
3. Define a higher-level `ModuleDefinition` for the toggle switch itself.
4. Instantiate both inverters inside the toggle switch module.
5. Use `MapsTo` to connect the inputs and outputs of the inverters, forming the complete, mutually-repressive feedback loop.

For more information on the MapsTo class and its properties, refer to page 30 in the SBOL 2.x.x specification document.

In [1]:
import sbol2

In [2]:
doc = sbol2.Document()

# Set the namespace for all SBOL objects
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')

## Define Shared Biological Parts

Here we define four `ComponentDefinition` objects representing the core parts of our toggle switch: the coding sequences for the two repressor proteins (LacI and cI) and the two promoters they regulate (pLac and pR). In a real design, these would be detailed with sequence information.

In [3]:
# RBS
B0032_rbs = sbol2.ComponentDefinition('B0032', sbol2.BIOPAX_DNA)
B0032_rbs.name = 'B0032_rbs'
B0032_rbs.addRole(sbol2.SO_RBS)
doc.addComponentDefinition(B0032_rbs)

# Terminator
B0015_terminator = sbol2.ComponentDefinition('B0015', sbol2.BIOPAX_DNA)
B0015_terminator.name = 'B0015_terminator'
B0015_terminator.addRole(sbol2.SO_TERMINATOR)
doc.addComponentDefinition(B0015_terminator)

In [4]:
# Coding Sequences (CDS)
lacI_cds = sbol2.ComponentDefinition('LacI_CDS', sbol2.BIOPAX_DNA)
lacI_cds.name = 'LacI Coding Sequence'
lacI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(lacI_cds)

cI_cds = sbol2.ComponentDefinition('cI_CDS', sbol2.BIOPAX_DNA)
cI_cds.name = 'Lambda cI Coding Sequence'
cI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(cI_cds)


# Promoters
pLac = sbol2.ComponentDefinition('pLac', sbol2.BIOPAX_DNA)
pLac.name = 'pLac Promoter'
pLac.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pLac)

pR = sbol2.ComponentDefinition('pR', sbol2.BIOPAX_DNA)
pR.name = 'pR Promoter'
pR.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pR)

In [5]:
# The toggle switch's behavior is defined by two protein pools
LacI_protein = sbol2.ComponentDefinition('LacI_protein', sbol2.BIOPAX_PROTEIN)
LacI_protein.name = 'LacI Protein'
LacI_protein.addRole('http://identifiers.org/ncit/NCIT:C17207') # 'Transcriptional factor'
doc.addComponentDefinition(LacI_protein)

cI_protein = sbol2.ComponentDefinition('cI_protein', sbol2.BIOPAX_PROTEIN)
cI_protein.name = 'Lambda cI Protein'
cI_protein.addRole('http://identifiers.org/ncit/NCIT:C17208') # 'Transcriptional factor'
doc.addComponentDefinition(cI_protein) 

## Define the Toggle Switch Module (Higher-Level Circuit)

This `ModuleDefinition` represents the complete genetic toggle switch. From this high-level perspective, we only care about the two key players: the LacI protein and the cI protein.

In [6]:
toggle_md = sbol2.ModuleDefinition('toggle_switch')

# Create functional components by calling using protein ComponentDefinitions
lacI_prot_fc = toggle_md.functionalComponents.create('LacI_protein_toggle')
lacI_prot_fc.definition = LacI_protein
lacI_prot_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

cI_prot_fc = toggle_md.functionalComponents.create('cI_protein_toggle')
cI_prot_fc.definition = cI_protein
cI_prot_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

## Define the Inverter Module Definition(Lower-Level Subsystems)

A toggle switch is composed of two inverters. We define each as a `ModuleDefinition`. Each inverter has an input (the repressor protein), a regulated promoter, and an output (the protein it produces). We also add `Interaction` objects to specify the biological function: repression.

- **LacI Inverter**: Takes LacI as input to repress the pLac promoter, which produces the cI protein.
- **cI Inverter**: Takes cI as input to repress the pR promoter, which produces the LacI protein.

In [7]:
# 1. LacI Inverter Module Definition (LacI represses production of cI)
lacI_inverter_md = sbol2.ModuleDefinition('LacI_Inverter')

lacI_inverter_plac_comp = sbol2.component.Component('pLac_promoter')
lacI_inverter_plac_comp.definition = pLac.identity
lacI_inverter_b0032_rbs = sbol2.component.Component('B0032_rbs')
lacI_inverter_b0032_rbs.definition = B0032_rbs.identity
lacI_inverter_cI_cds = sbol2.component.Component('cI_cds')
lacI_inverter_cI_cds.definition = cI_cds.identity
lacI_inverter_b0015_terminator = sbol2.component.Component('B0015_terminator')
lacI_inverter_b0015_terminator.definition = B0015_terminator.identity

## LacI Inverter Transcriptional Unit Component Definition
lacI_engineered_region = sbol2.ComponentDefinition('LacI_Engineered_Region', sbol2.BIOPAX_DNA)
lacI_engineered_region.name = 'LacI Engineered Region'
lacI_engineered_region.addRole('http://identifiers.org/so/SO:0000804')
lacI_engineered_region.components.add(lacI_inverter_plac_comp)
lacI_engineered_region.components.add(lacI_inverter_b0032_rbs)
lacI_engineered_region.components.add(lacI_inverter_cI_cds)
lacI_engineered_region.components.add(lacI_inverter_b0015_terminator)
doc.addComponentDefinition(lacI_engineered_region)

## pLac Promoter Functional Component
pLac_prom_fc_lacI_inverter = lacI_inverter_md.functionalComponents.create('pLac_promoter_LacI_inverter')
pLac_prom_fc_lacI_inverter.definition = lacI_inverter_plac_comp
pLac_prom_fc_lacI_inverter.direction = sbol2.SBOL_DIRECTION_NONE


## LacI Inverter Proteins Functional Component
lacI_prot_fc_lacI_inverter = lacI_inverter_md.functionalComponents.create('LacI_protein_LacI_inverter')
lacI_prot_fc_lacI_inverter.definition = LacI_protein
lacI_prot_fc_lacI_inverter.direction  = sbol2.SBOL_DIRECTION_IN

cI_prot_fc_lacI_inverter = lacI_inverter_md.functionalComponents.create('cI_protein_LacI_inverter')
cI_prot_fc_lacI_inverter.definition = cI_protein
cI_prot_fc_lacI_inverter.direction  = sbol2.SBOL_DIRECTION_OUT


# 2. cI Inverter Module (cI represses production of LacI)
cI_inverter_md = sbol2.ModuleDefinition('cI_Inverter')

cI_inverter_pR_comp = sbol2.component.Component('pR_promoter')
cI_inverter_pR_comp.definition = pR.identity
cI_inverter_b0032_rbs = sbol2.component.Component('B0032_rbs')
cI_inverter_b0032_rbs.definition = B0032_rbs.identity
cI_inverter_lacI_cds = sbol2.component.Component('lacI_cds')
cI_inverter_lacI_cds.definition = lacI_cds.identity
cI_inverter_b0015_terminator = sbol2.component.Component('B0015_terminator')
cI_inverter_b0015_terminator.definition = B0015_terminator.identity

## cI Inverter Transcriptional Unit Component Definition
cI_engineered_region = sbol2.ComponentDefinition('cI_Engineered_Region', sbol2.BIOPAX_DNA)
cI_engineered_region.name = 'cI Engineered Region'
cI_engineered_region.addRole('http://identifiers.org/so/SO:0000804') 
cI_engineered_region.components.add(cI_inverter_pR_comp)
cI_engineered_region.components.add(cI_inverter_b0032_rbs)
cI_engineered_region.components.add(cI_inverter_lacI_cds)
cI_engineered_region.components.add(cI_inverter_b0015_terminator)
doc.addComponentDefinition(cI_engineered_region)

## pR promoter Unit Functional Component
pR_prom_fc_cI_inverter = cI_inverter_md.functionalComponents.create('pR_promoter_cI_inverter')
pR_prom_fc_cI_inverter.definition = cI_inverter_pR_comp
pR_prom_fc_cI_inverter.direction = sbol2.SBOL_DIRECTION_NONE

## cI Inverter Proteins Functional Component
lacI_prot_fc_cI_inverter = cI_inverter_md.functionalComponents.create('LacI_protein_cI_inverter')
lacI_prot_fc_cI_inverter.definition = LacI_protein
lacI_prot_fc_cI_inverter.direction  = sbol2.SBOL_DIRECTION_OUT

cI_prot_fc_cI_inverter = cI_inverter_md.functionalComponents.create('cI_protein_cI_inverter')
cI_prot_fc_cI_inverter.definition = cI_protein
cI_prot_fc_cI_inverter.direction  = sbol2.SBOL_DIRECTION_IN

In [8]:
# Interaction 1.1: LacI protein inhibiting cI production
interaction1_1 = sbol2.Interaction('LacI_represses_pLac')
interaction1_1.interaction_type = sbol2.SBO_INHIBITION
lacI_inverter_md.interactions.add(interaction1_1)

# Creating participation objects
## LacI protein is the INHIBITOR
inhibitor1_1 = interaction1_1.participations.create('part_LacI_protein')
inhibitor1_1.participant = lacI_prot_fc_lacI_inverter
inhibitor1_1.roles = [sbol2.SBO_INHIBITOR]

## pLac promoter is being INHIBITED
inhibited1_1 = interaction1_1.participations.create('part_pLac_promoter')
inhibited1_1.participant = pLac_prom_fc_lacI_inverter
inhibited1_1.roles = [sbol2.SBO_INHIBITED]

# Interaction 1.2 : cI production
interaction1_2 = sbol2.Interaction('cI_production')
interaction1_2.interaction_type = sbol2.SBO_GENETIC_PRODUCTION
lacI_inverter_md.interactions.add(interaction1_2)

# Creating participation objects
## pLac is the promoter
prom_1_2 = interaction1_2.participations.create('part_pLac_promoter')
prom_1_2.participant = pLac_prom_fc_lacI_inverter
prom_1_2.roles = [sbol2.SBO_PROMOTER]

## cI is the product
product_1_2 = interaction1_2.participations.create('part_cI_protein')
product_1_2.participant = cI_prot_fc_lacI_inverter
product_1_2.roles = [sbol2.SBO_PRODUCT]





# Interaction 2.1: cI protein inhibiting LacI production
interaction2_1 = sbol2.Interaction('cI_represses_pR')
interaction2_1.interaction_type = sbol2.SBO_INHIBITION
cI_inverter_md.interactions.add(interaction2_1)

## cI protein is the INHIBITOR
inhibitor2_1 = interaction2_1.participations.create('part_cI_protein')
inhibitor2_1.roles = [sbol2.SBO_INHIBITOR]
inhibitor2_1.participant = cI_prot_fc_cI_inverter

## pR promoter is being INHIBITED
inhibited2_1 = interaction2_1.participations.create('part_transcriptional_unit')
inhibited2_1.roles = [sbol2.SBO_INHIBITED]
inhibited2_1.participant = pR_prom_fc_cI_inverter


# Interaction 2.2 : LacI production
interaction2_2 = sbol2.Interaction('lacI_production')
interaction2_2.interaction_type = sbol2.SBO_GENETIC_PRODUCTION
cI_inverter_md.interactions.add(interaction2_2)

# Creating participation objects
## pLac is the promoter
prom_2_2 = interaction2_2.participations.create('part_pR_promoter')
prom_2_2.participant = pR_prom_fc_cI_inverter
prom_2_2.roles = [sbol2.SBO_PROMOTER]

## cI is the product
product_2_2 = interaction2_2.participations.create('part_cI_protein')
product_2_2.participant = lacI_prot_fc_cI_inverter
product_2_2.roles = [sbol2.SBO_PRODUCT]

## Add Inverter Modules to Toggle Switch (Composition)

We now instantiate both inverter modules within the toggle switch module. This reflects **module composition** in SBOL—building a larger system from smaller, defined subsystems.

In [9]:
# Instantiate the LacI inverter
laci_inverter_instance = toggle_md.modules.create('laci_inverter_inst')
laci_inverter_instance.definition = lacI_inverter_md
laci_inverter_instance.displayId = 'laci_inverter_inst'

# Instantiate the cI inverter
ci_inverter_instance = toggle_md.modules.create('ci_inverter_inst')
ci_inverter_instance.definition = cI_inverter_md
ci_inverter_instance.displayId = 'ci_inverter_inst'

## Wire the Circuit with `MapsTo`

This is the key step. We use `MapsTo` objects to connect the components and form the feedback loop. We need four mappings to declare that:
1. The `LacI_protein` in the toggle switch is the same molecule that acts as the input to the LacI inverter.
2. The `cI_protein` in the toggle switch is the same molecule that is produced as the output of the LacI inverter.
3. The `cI_protein` in the toggle switch is the same molecule that acts as the input to the cI inverter.
4. The `LacI_protein` in the toggle switch is the same molecule that is produced as the output of the cI inverter.

This wiring correctly models the mutual repression: LacI represses cI production, and cI represses LacI production.

In [10]:
# Map 1: LacI protein is the input to the LacI inverter
map1 = laci_inverter_instance.mapsTos.create('map_laci_in_laci_protein')
map1.refinement = sbol2.SBOL_REFINEMENT_VERIFY_IDENTICAL
map1.local = lacI_prot_fc
map1.remote = lacI_prot_fc_lacI_inverter

# Map 2: cI protein is the output of the LacI inverter
map2 = laci_inverter_instance.mapsTos.create('map_ci_out_cI_protein')
map2.refinement = sbol2.SBOL_REFINEMENT_VERIFY_IDENTICAL
map2.local = cI_prot_fc
map2.remote = cI_prot_fc_lacI_inverter

# Map 3: cI protein is the input to the cI inverter
map3 = ci_inverter_instance.mapsTos.create('map_ci_in')
map3.refinement = sbol2.SBOL_REFINEMENT_VERIFY_IDENTICAL
map3.local = cI_prot_fc
map3.remote = cI_prot_fc_cI_inverter

# Map 4: LacI protein is the output of the cI inverter
map4 = ci_inverter_instance.mapsTos.create('map_laci_out')
map4.refinement = sbol2.SBOL_REFINEMENT_VERIFY_IDENTICAL
map4.local = lacI_prot_fc
map4.remote = lacI_prot_fc_cI_inverter

## Finalize and Validate

We add all high-level modules to the document and check that the complete design conforms to SBOL specifications. If valid, it is saved as `toggle_switch_complete.xml`.

In [11]:
doc.addModuleDefinition(lacI_inverter_md)
doc.addModuleDefinition(cI_inverter_md)
doc.addModuleDefinition(toggle_md)

# Validate the document
report = doc.validate()

if report == "Valid.":
    doc.write("example_mapsto.xml")
else:
    print(report)